# Space

In [ ]:
import os
import logging
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
# print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
import sys
from proj_space import SPACE
sys.path.append(SPACE['CODE_FN'])
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

from datasets import disable_caching
disable_caching()

SPACE['MODEL_ENDPOINT'] = 'vTest'

# Part 1: AIData

## Load AIData

In [ ]:
from recfldtkn.aidata_base.aidata import AIData

DATA_AIDATA = SPACE['DATA_AIDATA']
OneAIDataName = 'CgmLhm_Bf24Af2Af2t8H_5Min_3Cohort_EventFlt_Sample'

aidata = AIData.load_aidata(DATA_AIDATA, OneAIDataName, SPACE)
aidata

## Update EntryArgs

In [ ]:
OneEntryArgs = {
    # ----------------- Task Part -----------------
    'Task_Part': {

        'Tagging': {
            # 'TagName_to_TaggingMethod': {
            #     # TagName: TaggingMethod {Rules: [(x,x,x)], Op: and or}
            # },
            # 'ColumnsAddToDsCase': [],
            'TagFilter': True, # <--- still need to add Fitlter Tag, as we need to do the RandomDownSample.
            'TagSplit': False, # <--- do not need to add Split Tag anymore, as we already have. 
        },

        'Filtering': {
            # 'FilterTagging': None,
            'FilterTagging': {
                "Rules": [
                    ['RandDownSample', '<=', 0.5],
                    ['co.Bf24H_Food_recnum:recnum', '>=', 1], 
                    ], 
                'Op': 'and',
            }
        }, 
        
        'Splitting': {
            # 'SplitTagging': { # <----- for the Tagging part.
            #     'RANDOM_SEED': 32,
            #     'out_ratio': 0.1,
            #     'test_ratio': 'tail0.1',
            #     'valid_ratio': 0.1
            # },
            'TrainEvals': {
                'TrainSetName': 'In-Train', 
                'EvalSetNames': ['In-Test', 'In-Valid', 'Out']
            },
        }
    },

    # ----------------- Input Part -----------------
    'Input_Part': {
        'EntryInputMethod': 'Mto1Period_MultiTknInStep',
        'CF_list': [
            'cf.TargetCGM_Bf24H',
            'cf.TargetCGM_Af2H',

            'cf.ActivitySparse_Bf24H',
            'cf.ActivitySparse_Af2H',

            # 'cf.TimeSparse_Bf24H', 
            # 'cf.TimeSparse_Af2H',


            'cf.DietSparse_Bf24H',
            'cf.DietSparse_Af2H',
        ],
        'TargetField': 'TargetCGM',
        # 'TimeField':   'Time',
        'EventFields': [
            'Activity',
            'Diet',
        ],
        'BeforePeriods': ['Bf24H'],
        'AfterPeriods': ['Af2H'],
        'InferenceMode': False, # 'WithFutureEvent' #  # 'NoFutureEvent', 'WithFutureEvent', 
    }, 

    # ----------------- Output Part -----------------
    'Output_Part': {
        'EntryOutputMethod': 'EventPrediction',
        # 'MaskingRate': 0,
        'Task_Label': 'Diet',
        # other parameters toward X and Y value
        'agg_function':None,
        'label_process': None, 
    },
}


aidata.update_NameToData_with_OneEntryArgs(OneEntryArgs)

In [ ]:
Name_to_Data = aidata.Name_to_Data
Name = 'In-Train'

In [ ]:
aidata.Name_to_DS

In [ ]:
Data = aidata.Name_to_Data[Name]
ds_tfm = Data['ds_tfm']
ds_tfm

## View A Batch

In [ ]:
# aidata.Name_to_DsAIData
split_name = [i for i in  aidata.Name_to_Data][0]
dataset = aidata.Name_to_Data[split_name]
df_case = dataset['df_case']
df_case.head()

In [ ]:
ds_tfm = dataset['ds_tfm']
ds_tfm

In [ ]:
batch_size = 4
batch = ds_tfm[:batch_size]
batch

In [ ]:
batch['input_ids'][0] # .shape # 

In [ ]:
batch['labels'][0] # .shape # 